In [2]:
import pandas as pd
from pandas.io import sql
import sqlite3
import numpy as np

# read merged data from csv 
all_data = pd.read_csv("state_population.csv")

# calculate death rate
age_range="NULL"
ranges = np.array([0]+[(5+10*x) for x in range(1,8)]) # [ 0, 15, 25, 35, 45, 55, 65, 75]
ranges = np.hstack((ranges[:,None],(ranges + 10*np.ones(len(ranges)))[:,None]))
ranges[0,1] = ranges[1,0]
ranges[-1,-1] = -1
for rg in ranges:
    s, e = [int(i) for i in rg.tolist()]
    if e> 0:
        age_range = f"{s}_{e-1}"
    else:
        age_range = f"{s}+"
    all_data['drate_'+age_range] = all_data['death_'+age_range]/all_data[age_range]
all_data.head()

,Unnamed: 0,0_14,15_24,25_34,35_44,45_54,55_64,65_74,75+,all,...,rank_all,state,drate_0_14,drate_15_24,drate_25_34,drate_35_44,drate_45_54,drate_55_64,drate_65_74,drate_75+
0,0,82934.0,75579.0,71288.0,62205.0,74082.0,77479.0,53289.0,47792.0,544648.0,...,12.0,Rhode Island,0.0,0.0,0.000267,0.000322,0.000405,0.000232,0.000000,0.000000
1,1,82934.0,75579.0,71288.0,62205.0,74082.0,77479.0,53289.0,47792.0,544648.0,...,1.0,Rhode Island,0.0,0.0,0.000000,0.000000,0.000148,0.000568,0.001257,0.012157
2,2,82934.0,75579.0,71288.0,62205.0,74082.0,77479.0,53289.0,47792.0,544648.0,...,2.0,Rhode Island,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000281,0.005963
3,3,82934.0,75579.0,71288.0,62205.0,74082.0,77479.0,53289.0,47792.0,544648.0,...,3.0,Rhode Island,0.0,0.0,0.000000,0.000000,0.000000,0.000323,0.001295,0.003996
4,4,82934.0,75579.0,71288.0,62205.0,74082.0,77479.0,53289.0,47792.0,544648.0,...,4.0,Rhode Island,0.0,0.0,0.000000,0.000000,0.000000,0.000607,0.001764,0.002616


In [3]:
# load data into db
conn = sqlite3.connect(':memory:')
all_data.to_sql('Table1',conn)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [4]:
def getdata4user(conn, gender, state, age):
    age_range="NULL"
    ranges = np.array([0]+[(5+10*x) for x in range(1,8)]) # [ 0, 15, 25, 35, 45, 55, 65, 75]
    ranges = np.hstack((ranges[:,None],(ranges + 10*np.ones(len(ranges)))[:,None]))
    ranges[0,1] = ranges[1,0]
    ranges[-1,-1] = -1
    for rg in ranges:
        s, e = [int(i) for i in rg.tolist()]
        if age in range(s,e):
            if e> 0:
                age_range = f"{s}_{e-1}"
            else:
                age_range = f"{s}+"
    query = f"select name,drate_{age_range},death_{age_range},state,gender from Table1 where state=\'{state}\' and gender=\'{gender}\' order by drate_{age_range} desc;"
    print(query)
    return sql.read_sql_query(query, conn)

In [6]:
getdata4user(conn, "female", "Alabama", 15)

select name,drate_15_24,death_15_24,state,gender from Table1 where state='Alabama' and gender='female' order by drate_15_24 desc;


,name,drate_15_24,death_15_24,state,gender
0,Road Traffic Accidents,0.000156,50.0,Alabama,female
1,Homicide,0.000069,22.0,Alabama,female
2,Poisonings,0.000053,17.0,Alabama,female
3,Suicide,0.000044,14.0,Alabama,female
4,Coronary Heart Disease,0.000000,0.0,Alabama,female
5,Breast Cancer,0.000000,0.0,Alabama,female
6,Liver Disease,0.000000,0.0,Alabama,female
7,Diabetes Mellitus,0.000000,0.0,Alabama,female
8,Endocrine Disorders,0.000000,0.0,Alabama,female
9,Stroke,0.000000,0.0,Alabama,female


In [ ]:
import numpy as np

x = np.arange(5)
print(x)

In [ ]:
x + 10*np.ones(len(x))

In [ ]:
[0]+[(5+10*x) for x in range(1,8)]

In [ ]:
np.array([0]+[(5+10*x) for x in range(1,8)])

In [ ]:
ranges = np.array([0]+[(5+10*x) for x in range(1,8)]) # [ 0, 15, 25, 35, 45, 55, 65, 75]

In [ ]:
ranges = np.hstack((ranges[:,None],(ranges + 10*np.ones(len(ranges)))[:,None]))
ranges[0,1] = ranges[1,0]
ranges[-1,-1] = -1
print(ranges)

In [ ]:
for rg in ranges:
    print(rg)

In [ ]:
5.5 in range(1,10)

In [ ]:
range((1,2))

In [ ]:
a= (1,2)

In [ ]:
a[0]